In [2]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
#from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import os, json

In [3]:
train_dataset = load_from_disk("/opt/ml/data/train_dataset/train")
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")
#wiki_dataset = pd.read_json('../data/wikipedia_documents.json',orient='index') # wiki context
data_path = "../data/"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

wiki_dataset = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
) 


test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation')


In [4]:
valid_corpus = list(set(valid_dataset['context']))
train_corpus = list(set(train_dataset['context']))
wiki_corpus = list(set(wiki_dataset))
query = valid_dataset['question']
test_query = test_dataset['question']

In [5]:
#tf-idf
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
tokenizer_func = tokenizer.tokenize
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2),max_features=50000)
vectorizer.fit(wiki_corpus) # wiki_corpus 학습
sp_matrix = vectorizer.transform(wiki_corpus)
query_vec = vectorizer.transform(query)

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


In [13]:
print(query_vec.shape)
print(sp_matrix.shape)

(240, 50000)
(56737, 50000)


In [14]:
result = query_vec * sp_matrix.T # tf-idf-metrix


In [19]:
ground_truth_list = []
top_k_tfidf_valid = {}

for i in range(len(query)):

  idx = i

  passage_result = result[idx].data
  sorted_result = np.argsort(-passage_result)
  doc_scores = result[idx].data[sorted_result]
  doc_ids = result[idx].indices[sorted_result]
  q = query[idx]
  k = 150 # top_100까지 저장
  #print(ground_truth, "\n")
  temp_top_k = []
  for i in range(k):
    doc_id = doc_ids[i]
    temp_top_k.append(wiki_corpus[doc_id]) # retrieval
  top_k_tfidf_valid[q] = temp_top_k
  

In [20]:
# 저장
with open("/opt/ml/data/tfidf_fit_train.bin", "wb") as file:
    pickle.dump(top_k_tfidf_valid,file)


#### BM25 

In [21]:
from rank_bm25 import BM25Okapi

In [22]:
tokenized_corpus = [tokenizer.tokenize(context) for context in wiki_corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [30]:
top_k_bm25_valid = {}
count = 0
for q in query:
    top_k_bm25 = []
    encoded_query = tokenizer.tokenize(q)
    top_150 = bm25.get_top_n(encoded_query, wiki_corpus, n=200)
    count += 1
    if count%100 == 0:
        print(count)
    top_k_bm25_valid[q] = top_150

100
200


In [28]:
with open("/opt/ml/data/bm25_fit_train.bin", "wb") as file:
    pickle.dump(top_k_bm25_valid,file)

In [9]:
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("kykim/bert-kor-base" )

In [10]:
eval_batch_size = 32
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)
if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

# Construt dataloader
train_p_seqs = tokenizer(wiki_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(
    train_p_seqs["input_ids"],
    train_p_seqs["attention_mask"],
    train_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

torch.cuda.empty_cache()

Iteration: 100%|██████████| 1774/1774 [09:48<00:00,  3.02it/s]


In [11]:
train_q_seqs = tokenizer(
    query,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

query_dataset = TensorDataset(
    train_q_seqs["input_ids"],
    train_q_seqs["attention_mask"],
    train_q_seqs["token_type_ids"]
)

query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(
    query_dataset,
    sampler=query_sampler,
    batch_size=eval_batch_size
)

q_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        query_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    q_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        q_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = q_encoder(**q_inputs).to("cpu").numpy()
        q_embs.extend(outputs)

torch.cuda.empty_cache()
print('done')

Iteration: 100%|██████████| 8/8 [00:02<00:00,  3.27it/s]

done


In [12]:
p_embs = np.array(p_embs)
q_embs = np.array(q_embs)

print(p_embs.shape)
print(q_embs.shape)

(56737, 768)
(240, 768)


In [13]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

In [15]:
dense_p_retrieval_result = {}
idx = 0
for i in tqdm(range(len(query))):
    p_list = []
    q = query[i]
    for j in range(100):
        p_list.append(wiki_corpus[rank[idx][j]])
    dense_p_retrieval_result[q] = p_list
    idx += 1

100%|██████████| 240/240 [00:00<00:00, 461.38it/s]


In [16]:
#저장
with open("/opt/ml/data/dense_fit_train.bin", "wb") as file:
    pickle.dump(dense_p_retrieval_result,file)

In [39]:
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")
#top_k_list = [1 ,5, 10, 20, 50, 100, 150, 200]
top_k_list = [100]

wrong_idx = {}

for top_k in top_k_list:

    tfdif_acc = 0
    bm25_acc = 0
    for i in range(len(query)):
        q = query[i]
        ground_truth = valid_dataset['context'][i]

        flag = 0
        if ground_truth in top_k_tfidf_valid[q][:top_k]:
            tfdif_acc += 1
        if ground_truth in top_k_bm25_valid[q][:top_k]:
            bm25_acc += 1
            flag = 1
        if flag == 0:
            wrong_idx[i] = top_k_bm25_valid[q][:top_k]
    print('score_top_k : ', top_k)
    print('TF-IDF ACC : ', tfdif_acc / len(query))
    print('BM25 ACC : ', bm25_acc / len(query))
    print()


score_top_k :  100
TF-IDF ACC :  0.9041666666666667
BM25 ACC :  0.975



In [64]:
answer = valid_dataset['answers']
print(answer[0])
answer2 = [ans['text'][0] for ans in answer]
answer2

{'answer_start': [284], 'text': ['한보철강']}


['한보철강',
 '1871년',
 '나뭇잎',
 '금대야',
 '수평적 관계',
 '옥음방송',
 '코칭 스티치',
 '복잡한 감염병',
 '스페인',
 '20세기 초',
 '"5월의 왕"',
 "'일급 비밀 프로젝트 2501'",
 '테헤란',
 '역사교육과정개발추진위원회',
 '1967년 11월 15일',
 '1965년',
 '아리크 부케',
 '〈중앙일보〉',
 '미타케성',
 '전체 4순위',
 '뇌물',
 '보통 유형 준융합성 천연두',
 '데스탱 장군',
 '박트리아',
 "'진전(陳田)'이라 새겨진 기와조각",
 '《국가》',
 '2011년 3월 19일',
 '유럽 포르투갈어',
 '메이저 릿지',
 '요희',
 '포드 극장',
 '만주국 관리',
 '『협동조합에 관하여』',
 '도고쿠',
 '몽키 D. 가프',
 '조류',
 '광주교도소',
 '작가 베게티우스',
 '가오슝 시',
 '쇠망치',
 '마리즈 교수',
 '런던',
 '이틀',
 '의상대사',
 '홋카이도',
 '‘우유의 바다’',
 '장어',
 '소련',
 '전치',
 '흑색육(黑色肉)',
 '욱',
 '독일인민당',
 '하기노 역',
 '회칠',
 '황강다리',
 '직업 교육',
 '공산당',
 '매년 음력 정월',
 '조계종',
 '베이징',
 '독일군',
 '1932년',
 '제서지전(齊西之戰)이후',
 '가루아',
 '출생 천궁도',
 '나말여초',
 '1939년',
 '창 절제술',
 '도버 밀',
 '여정현(呂正鉉)',
 '광배',
 '울산',
 '땅(은색)과 하늘(금색)을 맺고 푸는 권한',
 '저수지',
 '퀘이커 교',
 '피에르 오주로',
 '지분',
 '독일',
 '제 3자',
 '페이팔',
 '가리타 히사노리',
 '점수',
 '코스모케라톱스',
 "'사채회사'",
 '스위스',
 '"공유지의 비극"',
 '그 자체가 계급 모순을 가지는 체제',
 '해리 트루먼 대통령',
 '무위태수',
 '크라운라이터',
 '1990년 9월',
 '"인간과 요괴의 완전한 평

In [41]:
wrong_idx.keys()

dict_keys([43, 63, 148, 168, 204, 232])

In [52]:
v_context = valid_dataset['context']
for k, v in wrong_idx.items():
    print("질문", query[k],"답",answer[k])
    print("ground_truth:", v_context[k])
    print()
    print("most_similar:", v[0])
    print()

질문 국내 화엄종의 선구자는 누구인가? 답 의상대사
ground_truth: 이 벽화는 부석사를 창건하고 우리나라에서 화엄종을 처음 시작한 의상대사를 모시고 있는 부석사조사당(국보 제19호) 안쪽 벽면에 사천왕과 제석천, 범천을 6폭으로 나누어 그린 그림이다. 지금은 벽화가 있는 벽면 전체를 그대로 떼어 유리상자에 담아 무량수전에 보관하고 있다.\n\n흙벽 위에 녹색으로 바탕을 칠하고 붉은색·백색·금색 등으로 채색하였으며, 각각의 크기는 길이 205 cm, 폭 75 cm 가량이다. 양쪽의 두 천부상은 우아한 귀족풍으로 양감이 풍만하며, 가운데 사천왕은 악귀를 밟고 서서 무섭게 노려 보는 건장한 모습이다. 훼손된 부분이 많고 후대에 덧칠하여 원래의 모습이 많이 사라졌지만 율동감 넘치는 유려한 선에서 고려시대 불화의 품격을 느낄 수 있다.\n\n건물에서 발견된 기록을 통해 조사당을 세운 연대가 고려 우왕 3년(1377)임을 알게 되었으며, 벽화를 그린 연대도 같은 시기일 것으로 보인다. 현재 우리나라에 남아 있는 벽화 가운데 가장 오래된 작품으로 회화사적으로 매우 중요한 작품으로 평가받고 있다.

most_similar: 영주 부석사 고려목판(榮州 浮石寺 高麗木板)은 부석사에 있는『화엄경』정원본 40권, 진본 60권, 주본 80권 등 3종의 대방광불화엄경을 나무판에 새긴 것이다. 대방광불화엄경은 줄여서 ‘화엄경’이라 부르기도 하는데 화엄종의 근본경전으로 법화경과 함께 한국 불교사상 확립에 가장 크게 영향을 끼친 경전이다.

정원본(貞元本)은 반야가 번역한 화엄경 40권을 가리키고, 진본(晋本)은 불타발타라가 번역한 화엄경 60권을, 주본(周本)은 실차난타가 번역한 화엄경 80권을 가리킨다. 이 판은 모두 합쳐 634판이며 한 줄에 34자씩 글자를 배열한 특이한 형식이다.

13∼14세기경 거란에서 불경을 수입하여 나무판에 다시 새긴 것으로 추정되며, 현재 전해지는 유일한 거란본 계열의 각판으로 매우 중요한 자료이다. 우리나라 화엄종의 창시자인 의상대사가 창건하여 

In [56]:
test = top_k_bm25_valid[query[0]][:5]

In [57]:
import re

In [60]:
t = "나는    정진원이다."
t = re.sub(r"\s+", " ", t)
print(t)
#for t in test:
#    t = re.sub(r"\s+", " ", t)
#    print(t)

나는 정진원이다.


In [65]:
answer[10]

{'answer_start': [274], 'text': ['"5월의 왕"']}